In [1]:
import os
import h5py
import time
import warnings
import datetime
import numpy as np

from featureExtraction.combination.ortho_tf import *

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score

In [2]:
files_clean = os.listdir("../Dataset/Clean")
files_templated = os.listdir("../Dataset/Raw")

clean = [x for x in files_clean if (x.lower().endswith(".csv"))]
clean.sort()

DATA_CLEAN = ["../Dataset/Clean/" + x for x in files_clean if (x.lower().endswith(".csv"))]
DATA_CLEAN.sort()
len_clean = len(DATA_CLEAN)

templated = [x for x in files_templated if (x.lower().endswith(".csv"))]
templated.sort()

DATA_TEMPLATED = ["../Dataset/Raw/" + x for x in files_templated if (x.lower().endswith(".csv"))]
DATA_TEMPLATED.sort()
len_templated = len(DATA_TEMPLATED)

range_len = max(len_clean, len_templated)

In [3]:
warnings.filterwarnings("ignore")

count = 1
fail = 0

f1_list = []
acc_list = []

In [4]:
for i in range(range_len):
    try:
        print("Running {}/{} ({})".format(count, len_clean, clean[count-1]))
        start_time = time.time()
        
        # Feature Extraction
        result_fe, label, feat_name = ortho_tf(clean[i])
        fe_time = time.time()

        # prepare classifier
        cv = KFold(n_splits=20, random_state=1, shuffle=True)
        logRes = LogisticRegression(random_state=1, max_iter=10000)
        
        # prepare the cross-validation procedure
        acc_score = cross_val_score(logRes, result_fe, label, scoring='f1_micro', cv=cv, n_jobs=-1)
        f1_score = cross_val_score(logRes, result_fe, label, scoring='f1_macro', cv=cv, n_jobs=-1)
        cl_time = time.time()

        print('Accuracy: {} ({:.5f})'.format(np.mean(acc_score), np.std(acc_score)))
        print('F1-Score: {} ({:.5f})'.format(np.mean(f1_score), np.std(f1_score)))
        print('FE Time: {} seconds'.format(fe_time - start_time))
        print('CL time: {} seconds'.format(cl_time - fe_time))
        
        f1_list.append(np.mean(f1_score))
        acc_list.append(np.mean(acc_score))
        count += 1
    except Exception as e:
        count += 1
        fail += 1
        print(i, ": ", e)
        pass
    print("================================================")

print("================================================")
print("Overall Accuracy = {} ({})".format(sum(acc_list)/len(acc_list), np.std(acc_list)))
print("Overall F1-Score = {} ({})".format(sum(f1_list)/len(f1_list), np.std(f1_list)))
print("Failed = {}/{}".format(fail, len_clean))

Running 1/13 (templated_posneg_Dataset_C_HT_4000.csv)
Accuracy: 0.9774999999999998 (0.00994)
F1-Score: 0.9773737032869796 (0.01004)
FE Time: 0.23991990089416504 seconds
CL time: 104.41428112983704 seconds
Running 2/13 (templated_posneg_KicauanBerlabel.csv)
Accuracy: 0.8750833333333334 (0.06236)
F1-Score: 0.8626036825143547 (0.07278)
FE Time: 0.03621268272399902 seconds
CL time: 6.311493635177612 seconds
Running 3/13 (templated_posneg_dataset_komentar_instagram_cyberbullying.csv)
Accuracy: 0.854078947368421 (0.07221)
F1-Score: 0.847952841138162 (0.07543)
FE Time: 0.03545522689819336 seconds
CL time: 4.333842754364014 seconds
Running 4/13 (templated_posneg_dataset_tweet_sentimen_tayangan_tv.csv)
Accuracy: 0.835 (0.06538)
F1-Score: 0.8233812522927983 (0.06980)
FE Time: 0.018159151077270508 seconds
CL time: 2.1602742671966553 seconds
Running 5/13 (templated_posneg_dataset_tweet_sentiment_cellular_service_provider.csv)
Accuracy: 0.8245238095238095 (0.08397)
F1-Score: 0.8112931952210454 (0.0